## Goal: Create a Combined Database with Mean Age of Maternal Mortality and GDP per Region in Mexico.
- This requires the combination of res_dataset (with each Region mean, variance and standard deviation), dates (contains Region and Metropolitan Area), and mexico_gdp (GDP by Metropolitan Area) subdataset. 

In [24]:
# Import the relevant python libraries for the analysis
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import scipy.stats as stats
import pylab as pl
import math

In [25]:
# Load and test dataset and set the index if applicable
metro_by_region = pd.read_excel('../data/metro_area_by_state.xlsx')
metro_by_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 5 columns):
Name               74 non-null object
Status             74 non-null object
State(s)           74 non-null object
Population 2010    74 non-null float64
Population 2015    74 non-null float64
dtypes: float64(2), object(3)
memory usage: 3.0+ KB


### Step 1: Explore the Data and Create Clean Sub-Dataframe for Analysis

In [26]:
metro_by_region

,Name,Status,State(s),Population 2010,Population 2015
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Acapulco,Metropolitan Area,Guerrero,863431.0,886975.0
3,Acayucan,Metropolitan Area,Veracruz de Ignacio de la Llave,112996.0,120340.0
4,Aguascalientes,Metropolitan Area,Aguascalientes,932369.0,1044049.0
5,Campeche,Metropolitan Area,Campeche,259005.0,283025.0
6,Cancún,Metropolitan Area,Quintana Roo,677379.0,763121.0
7,Celaya,Metropolitan Area,Guanajuato,690442.0,731667.0
8,Chetumal,Metropolitan Area,Quintana Roo,207810.0,224080.0
9,Chihuahua,Metropolitan Area,Chihuahua,852533.0,918339.0


##### Remove Irregular Values
- Case 1: Remove all rows with 'NaN' in the column values.

In [27]:
#Drop NaN values 
metro_by_region = metro_by_region.dropna()
metro_by_region.head()

,Name,Status,State(s),Population 2010,Population 2015
2,Acapulco,Metropolitan Area,Guerrero,863431.0,886975.0
3,Acayucan,Metropolitan Area,Veracruz de Ignacio de la Llave,112996.0,120340.0
4,Aguascalientes,Metropolitan Area,Aguascalientes,932369.0,1044049.0
5,Campeche,Metropolitan Area,Campeche,259005.0,283025.0
6,Cancún,Metropolitan Area,Quintana Roo,677379.0,763121.0


In [28]:
# Organize dataset by State in alphabetical order
metro_by_region = metro_by_region.sort_values(by=['State(s)'],ascending=True)
metro_by_region.head()

,Name,Status,State(s),Population 2010,Population 2015
4,Aguascalientes,Metropolitan Area,Aguascalientes,932369.0,1044049.0
64,Tijuana,Metropolitan Area,Baja California,1751430.0,1840710.0
20,Ensenada,Metropolitan Area,Baja California,466814.0,486639.0
34,Mexicali,Metropolitan Area,Baja California,936826.0,988417.0
28,La Paz,Metropolitan Area,Baja California Sur,251871.0,272711.0


In [29]:
# Reset index 
metro_by_region = metro_by_region.reset_index()
metro_by_region.head()

,index,Name,Status,State(s),Population 2010,Population 2015
0,4,Aguascalientes,Metropolitan Area,Aguascalientes,932369.0,1044049.0
1,64,Tijuana,Metropolitan Area,Baja California,1751430.0,1840710.0
2,20,Ensenada,Metropolitan Area,Baja California,466814.0,486639.0
3,34,Mexicali,Metropolitan Area,Baja California,936826.0,988417.0
4,28,La Paz,Metropolitan Area,Baja California Sur,251871.0,272711.0


### Since population is not the focus of this analysis, population columns can be removed.

In [30]:
# Remove unnecessary columns 
metro_by_region = metro_by_region.drop(['index', 'Population 2010', 'Population 2015'], axis=1)
metro_by_region.head()

,Name,Status,State(s)
0,Aguascalientes,Metropolitan Area,Aguascalientes
1,Tijuana,Metropolitan Area,Baja California
2,Ensenada,Metropolitan Area,Baja California
3,Mexicali,Metropolitan Area,Baja California
4,La Paz,Metropolitan Area,Baja California Sur


In [37]:
# Open dates as a global variable that can be uploaded to other Jupyter Notebooks
%store -r dates

In [38]:
print('There are '+ str(len(np.unique(dates['Residence Name']))) + ' Provinces in Mexico.')
list(np.unique(dates['Residence Name']))

There are 32 Provinces in Mexico.


['Aguascalientes',
 'Baja California',
 'Baja California Sur',
 'Campeche',
 'Chiapas',
 'Chihuahua',
 'Coahuila de Zaragoza',
 'Colima',
 'Distrito Federal',
 'Durango',
 'Guanajuato',
 'Guerrero',
 'Hidalgo',
 'Jalisco',
 'Michoacán de Ocampo',
 'Morelos',
 'México',
 'Nayarit',
 'Nuevo León',
 'Oaxaca',
 'Puebla',
 'Querétaro Arteaga',
 'Quintana Roo',
 'San Luis Potosí',
 'Sinaloa',
 'Sonora',
 'Tabasco',
 'Tamaulipas',
 'Tlaxcala',
 'Veracruz de Ignacio de la Llave',
 'Yucatán',
 'Zacatecas']

### Group Metropolitan Areas in by State

In [35]:
# Assess the number of Mexico States in dataset
print('The Number of Mexican States in this Dataset is: ',len(metro_by_region['State(s)'].unique()))
list(metro_by_region['State(s)'].unique())

The Number of Mexican States in this Dataset is:  37


['Aguascalientes',
 'Baja California',
 'Baja California Sur',
 'Campeche',
 'Chiapas',
 'Chihuahua',
 'Ciudad de México / Hidalgo / México',
 'Coahuila de Zaragoza',
 'Coahuila de Zaragoza / Durango',
 'Colima',
 'Durango',
 'Guanajuato',
 'Guanajuato / Michoacán de Ocampo',
 'Guerrero',
 'Hidalgo',
 'Jalisco',
 'Jalisco / Nayarit',
 'Michoacán de Ocampo',
 'Morelos',
 'México',
 'Nayarit',
 'Nuevo León',
 'Oaxaca',
 'Puebla',
 'Puebla / Tlaxcala',
 'Querétaro / Guanajuato',
 'Quintana Roo',
 'San Luis Potosí',
 'Sinaloa',
 'Sonora',
 'Tabasco',
 'Tamaulipas',
 'Tamaulipas / Veracruz de Ignacio de la Llave',
 'Tlaxcala',
 'Veracruz de Ignacio de la Llave',
 'Yucatán',
 'Zacatecas']

In [49]:
# Open dates as a global variable that can be uploaded to other Jupyter Notebooks
%store -r mexico_gdp

In [59]:
print('There are '+ str(len(np.unique(mexico_gdp['Metro Areas']))) + ' Metropolitan Areas in Mexico.')
metro_areas = mexico_gdp['Metro Areas'].sort_values()
list(metro_areas.unique())

There are 64 Metropolitan Areas in Mexico.


['Acapulco de Juarez',
 'Aguascalientes',
 'Ahome',
 'Benito Juarez',
 'Cajeme',
 'Campeche',
 'Celaya',
 'Centro',
 'Chihuahua',
 'Chilpancingo de los Bravo',
 'Coatzacoalcos',
 'Colima',
 'Cordoba',
 'Cuautla',
 'Cuernavaca',
 'Culiacan',
 'Durango',
 'Ensenada',
 'Guadalajara',
 'Guadalupe',
 'Hermosillo',
 'Irapuato',
 'Juarez',
 'La Paz',
 'Leon',
 'Los Cabos',
 'Matamoros',
 'Mazatlan',
 'Merida',
 'Mexicali',
 'Mexico',
 'Mexico City',
 'Minatitlan',
 'Monclova',
 'Monterrey',
 'Morelia',
 'Nuevo Laredo',
 'Oaxaca de Juarez',
 'Orizaba',
 'Pachuca de Soto',
 'Poza Rica de Hidalgo',
 'Puebla',
 'Puerto Vallarta',
 'Queretaro',
 'Reynosa',
 'Salamanca',
 'Saltillo',
 'San Juan del Rio',
 'San Luis Potosi',
 'Tampico',
 'Tapachula',
 'Tehuacan',
 'Tepic',
 'Tijuana',
 'Tlaxcala',
 'Toluca',
 'Torreon',
 'Tulancingo de Bravo',
 'Tuxtla Gutierrez',
 'Uruapan',
 'Veracruz',
 'Victoria',
 'Xalapa',
 'Zamora']

In [60]:
# Assess the number of Mexico Metro Areas in dataset
print('The Number of Mexican Matropolicatn Areas in this Dataset is: ',len(metro_by_region['Name'].unique()))
metro_areas_subdf = metro_by_region['Name'].sort_values()
list(metro_areas_subdf.unique())

The Number of Mexican Matropolicatn Areas in this Dataset is:  74


['Acapulco',
 'Acayucan',
 'Aguascalientes',
 'Campeche',
 'Cancún',
 'Celaya',
 'Chetumal',
 'Chihuahua',
 'Chilpancingo',
 'Ciudad Victoria',
 'Coatzacoalcos',
 'Colima - Villa de Álvarez',
 'Cuautla',
 'Cuernavaca',
 'Culiacán',
 'Córdoba',
 'Delicias',
 'Durango',
 'Ensenada',
 'Guadalajara',
 'Guanajuato',
 'Guaymas',
 'Hermosillo',
 'Hidalgo del Parral',
 'Juárez',
 'La Laguna\xa0(Comarca Lagunera, Torreón - Gómez Palacio)',
 'La Paz',
 'La Piedad - Pénjamo',
 'León',
 'Matamoros',
 'Mazatlán',
 'Mexicali',
 'Minatitlán',
 'Monclova - Frontera',
 'Monterrey',
 'Morelia',
 'Moroleón - Uriangato',
 'Mérida',
 'Nogales',
 'Nuevo Laredo',
 'Oaxaca',
 'Ocotlán',
 'Orizaba',
 'Pachuca',
 'Piedras Negras',
 'Poza Rica',
 'Puebla - Tlaxcala',
 'Puerto Vallarta',
 'Querétaro',
 'Reynosa',
 'Ríoverde',
 'Saltillo',
 'San Francisco del Rincón',
 'San Luis Potosí',
 'Tampico',
 'Tapachula',
 'Tecomán',
 'Tehuacán',
 'Tehuantepec\xa0(- Salina Cruz)',
 'Tepic',
 'Teziutlán',
 'Tianguistenco',


### In preparation for dataframe combination, the following cases must be resolved:

#### Case 1 
Some of the Metropolitan Area names need to be cleaned. Namely:
- 'La Laguna\xa0(Comarca Lagunera, Torreón - Gómez Palacio)' = 'Torreon'
- 'La Piedad - Pénjamo' = (not found in other dataset)
- 'Monclova - Frontera' = 'Monclova'
- 'Moroleón - Uriangato' = (not found in other dataset)
- 'Tehuantepec\xa0(- Salina Cruz)' = (not found in other dataset)
- 'Tlaxcala - Apizaco' = 'Tlaxcala'
- 'Valle de México\xa0[Greater Mexico City]' = 'Mexico City'
- 'Zacatecas - Guadalupe' = 'Guadalupe'

#### Case 2
There are 31 states and 1 federal entity in Mexico, so the length of the 'Regions' column should be 32, not 37. When looking over the list of regions within the dataset, it becomes apparent that there are duplicate entries of States within Mexico, and need to be combined together. Namely:
- 'Coahuila de Zaragoza' - 'Coahuila de Zaragoza / Durango'
- 'Guanajuato' - 'Guanajuato / Michoacán de Ocampo'
- 'Jalisco' - 'Jalisco / Nayarit'
- 'Puebla' - 'Puebla / Tlaxcala'
- 'Tamaulipas' - 'Tamaulipas / Veracruz de Ignacio de la Llave'

#### Case 1: Fix Metropolitan Names in metro_by_region dataset
- The values not found in the mexico_gdp dataset will be skipped.

In [91]:
# Replace the 5 metropolitan region names with the matching name in the GDP dataset

metro_by_region = metro_by_region.replace('La Laguna\xa0(Comarca Lagunera, Torreón - Gómez Palacio)', 'Torreon')
metro_by_region = metro_by_region.replace('Monclova - Frontera', 'Monclova')
metro_by_region = metro_by_region.replace('Tlaxcala - Apizaco', 'Tlaxcala')
metro_by_region = metro_by_region.replace('Valle de México\xa0[Greater Mexico City]', 'Mexico City')
metro_by_region = metro_by_region.replace('Zacatecas - Guadalupe', 'Guadalupe')

In [92]:
metro_areas_subdf_2 = metro_by_region['Name'].sort_values()
list(metro_areas_subdf_2.unique())

['Acapulco',
 'Acayucan',
 'Aguascalientes',
 'Campeche',
 'Cancún',
 'Celaya',
 'Chetumal',
 'Chihuahua',
 'Chilpancingo',
 'Ciudad Victoria',
 'Coatzacoalcos',
 'Colima - Villa de Álvarez',
 'Cuautla',
 'Cuernavaca',
 'Culiacán',
 'Córdoba',
 'Delicias',
 'Durango',
 'Ensenada',
 'Guadalajara',
 'Guadalupe',
 'Guanajuato',
 'Guaymas',
 'Hermosillo',
 'Hidalgo del Parral',
 'Juárez',
 'La Paz',
 'La Piedad - Pénjamo',
 'León',
 'Matamoros',
 'Mazatlán',
 'Mexicali',
 'Mexico City',
 'Minatitlán',
 'Monclova',
 'Monterrey',
 'Morelia',
 'Moroleón - Uriangato',
 'Mérida',
 'Nogales',
 'Nuevo Laredo',
 'Oaxaca',
 'Ocotlán',
 'Orizaba',
 'Pachuca',
 'Piedras Negras',
 'Poza Rica',
 'Puebla - Tlaxcala',
 'Puerto Vallarta',
 'Querétaro',
 'Reynosa',
 'Ríoverde',
 'Saltillo',
 'San Francisco del Rincón',
 'San Luis Potosí',
 'Tampico',
 'Tapachula',
 'Tecomán',
 'Tehuacán',
 'Tehuantepec\xa0(- Salina Cruz)',
 'Tepic',
 'Teziutlán',
 'Tianguistenco',
 'Tijuana',
 'Tlaxcala',
 'Toluca',
 'Torr

#### Case 2: Fix Region Names in metro_by_region dataset

In [66]:
# Create a function to combine all Metro Areas per Region 
coa_metros = metro_by_region[metro_by_region['State(s)'] == 'Coahuila de Zaragoza']['Name']
coa_metros_2 = metro_by_region[metro_by_region['State(s)'] == 'Coahuila de Zaragoza / Durango']['Name']
coa_combo = coa_metros.append(coa_metros_2)

In [62]:
# Combine the Metro Areas per Region *manually* in a List age per Regions with 2 entries
coa_metros = list(metro_by_region[metro_by_region['State(s)'] == 'Coahuila de Zaragoza']['Name']) + list(metro_by_region[metro_by_region['State(s)'] == 'Coahuila de Zaragoza / Durango']['Name'])
guana_metros = list(metro_by_region[metro_by_region['State(s)'] == 'Guanajuato']['Name']) + list(metro_by_region[metro_by_region['State(s)'] == 'Guanajuato / Michoacán de Ocampo']['Name'])
jali_metros = list(metro_by_region[metro_by_region['State(s)'] == 'Jalisco']['Name']) + list(metro_by_region[metro_by_region['State(s)'] == 'Jalisco / Nayarit']['Name'])
pueb_metros = list(metro_by_region[metro_by_region['State(s)'] == 'Puebla' ]['Name']) + list(metro_by_region[metro_by_region['State(s)'] == 'Puebla / Tlaxcala']['Name'])
tama_metros = list(metro_by_region[metro_by_region['State(s)'] == 'Tamaulipas' ]['Name']) + list(metro_by_region[metro_by_region['State(s)'] == 'Tamaulipas / Veracruz de Ignacio de la Llave']['Name'])

In [67]:
coa_combo

13                                       Piedras Negras
14                                             Saltillo
15                                  Monclova - Frontera
16    La Laguna (Comarca Lagunera, Torreón - Gómez P...
Name: Name, dtype: object

In [ ]:
# Create a dict variable to hold new key:value pairs
metro_region_updates = {}

metro_region_updates['Coahuila de Zaragoza'] = []
metro_region_updates['Guanajuato'] = []
metro_region_updates['Jalisco'] = []
metro_region_updates['Puebla'] = []
metro_region_updates['Tamaulipas'] = []

In [21]:
# Remove region key dupilcates (ones merged in section above)
metro_region_dict.pop('Coahuila de Zaragoza / Durango')
metro_region_dict.pop('Guanajuato / Michoacán de Ocampo')
metro_region_dict.pop('Jalisco / Nayarit')
metro_region_dict.pop('Puebla / Tlaxcala')
metro_region_dict.pop('Tamaulipas / Veracruz de Ignacio de la Llave')

array(['Tampico'], dtype=object)

In [22]:
len(metro_region_dict)

32

In [26]:
metro_region_dict

{'Aguascalientes': array(['Aguascalientes'], dtype=object),
 'Baja California': array(['Tijuana', 'Ensenada', 'Mexicali'], dtype=object),
 'Baja California Sur': array(['La Paz'], dtype=object),
 'Campeche': array(['Campeche'], dtype=object),
 'Chiapas': array(['Tuxtla Gutiérrez', 'Tapachula'], dtype=object),
 'Chihuahua': array(['Delicias', 'Juárez', 'Chihuahua', 'Hidalgo del Parral'],
       dtype=object),
 'Ciudad de México / Hidalgo / México': array(['Valle de México\xa0[Greater Mexico City]'], dtype=object),
 'Coahuila de Zaragoza': array(['Piedras NegrasLa Laguna\xa0(Comarca Lagunera, Torreón - Gómez Palacio)',
        'SaltilloLa Laguna\xa0(Comarca Lagunera, Torreón - Gómez Palacio)',
        'Monclova - FronteraLa Laguna\xa0(Comarca Lagunera, Torreón - Gómez Palacio)'],
       dtype=object),
 'Colima': array(['Tecomán', 'Colima - Villa de Álvarez'], dtype=object),
 'Durango': array(['Durango'], dtype=object),
 'Guanajuato': array(['Moroleón - UriangatoLa Piedad - Pénjamo',
    

In [23]:
# Store new dictionary as a global variable that can be uploaded to other Jupyter Notebooks
%store metro_region_dict

Stored 'metro_region_dict' (dict)


In [ ]:
# split string method after space in 'MEX ##: ' to choose the region name
# if no match for region name, skip
# if match for region name, store value per year in year variable
# for each string match per region, add value to year variable then divide sum by len(#_matches)
# add year var to region_year as key (dictionary)
# print dictionary as a bar chart 
# plot dictionary region up/down in a boxplot 